In [1]:
import scipy as sp
import commpy as cp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LinearLocator
from matplotlib import cm
from matplotlib import ticker

# import ray
# ray.init()

ModuleNotFoundError: No module named 'commpy'

In [ ]:
t_sim = 7*(10**-8)                  # длительность одного символа последовательности
T_p = 13.65 * (10**-6)              # период повторения импульсов
T_sl = T_p * 5                      # время стояния луча
dR = 10.5                           # разрешение по дальности
dV = 219.78                         # разрешение по скорости

dt_z = 2*dR/3e8                     # разрешение по времени задержки (оно же по дальности, но во временной области)

N = 2                               # количество импульсов
l = 15                              # длина последовательности

D = 0.91                            # правильное обнаружение
F = 4.1e-8                          # ложная тревога

F_main = 10e10                      # несущая частота
lambda_main = 3e8/F_main            # несущая длина волны
F_prom = 10/t_sim                   # промежуточная частота
lambda_prom = 3e8/F_prom            # длина волны промежуточная

print("Промежуточная частота:", F_prom)
print("Полоса:", 1/t_sim)
print("Промежуточная длина волны:", lambda_prom)

fd = 10*F_prom                      # частота дискретизации
td = 1/fd                           # период дискретизации

sigma_noise = np.sqrt((N*np.floor(t_sim*l/td))/((np.log10(F)/np.log10(D))-1))   # Уровень шума

print("SNR [разы] = ", (np.log10(F)/np.log10(D))-1)
print("Sigma noise = ", sigma_noise)

t_one = np.arange(0, ((t_sim*l)//td) * td, td)          # ось времени одного импульса
t_pach = np.arange(0, (T_sl//td) * td, td)              # ось времени одного стояния луча

n_t_sim = int(np.ceil(t_sim/td))
print(n_t_sim)
print("Время стояния луча: ", T_sl)

: 

### Входные данные о цели для имитации работы

In [ ]:
# Входные данные для имитации цели

R_str = input("\nВведите дальность до цели [м]: ")
V_str = input("Введите скорость цели [м/с]: ")

R = float(R_str)
V = float(V_str)

# параметры отраженного сигнала

t_z = 2*R/3e8               # время задержки сигнала до цели
f_sm = 2*V/lambda_main      # смещение частоты отраженного от цели сигнала
# f_sm = 2*V/lambda_prom      # смещение частоты отраженного от цели сигнала

print("\nВремя задержки:", t_z)
print("Смещение частоты:", f_sm)

: 

### Зондирующий сигнал

In [ ]:
# Последовательность Задова-Чу

ZC_root = cp.sequences.zcsequence(1, l)

CV = [2, 5, 8]

ZC_cv = np.zeros((len(CV), l), dtype=np.complex_)
i = 0
for cv in CV:
    for k in range(l):
        ZC_cv[i][k] = ZC_root[np.mod(k + cv, l)]
    i += 1

ZC = np.hstack([ZC_cv[0][:5], ZC_cv[1][:4], ZC_cv[2][:6]])

print(len(ZC))
print(ZC)

: 

In [ ]:
# Ортогональная последовательность Задова-Чу

ZC_root_ort = cp.sequences.zcsequence(2, l, q=2)

CV = [2, 5, 8]

ZC_cv_ort = np.zeros((len(CV), l), dtype=np.complex_)
i = 0
for cv in CV:
    for k in range(l):
        ZC_cv_ort[i][k] = ZC_root_ort[np.mod(k + cv, l)]
    i += 1

ZC_ort = np.hstack([ZC_cv_ort[0][:5], ZC_cv_ort[1][:4], ZC_cv_ort[2][:6]])

print(len(ZC_ort))
print(ZC_ort)

: 

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(17,10))

ax1.plot(range(l*10), [np.angle(ZC[i//10])/np.pi for i in range(150)])
ax1.set_ylim((-1.25, 1.25))
ax1.set_title("Модулирующая ZC-последовательность")

ax2.plot(range(l*10), [np.angle(ZC_ort[i//10])/np.pi for i in range(150)])
ax2.set_ylim((-1.25, 1.25))
ax2.set_title("Модулирующия ортогональная ZC-последовательность")

: 

In [ ]:
# ФКМ сформированный ZC-последовательностью

ZC_signal = np.zeros(len(t_one), dtype=np.complex_)

for i in range(len(t_one)):
    ZC_signal[i] = ZC[i//n_t_sim]

ZC_angle = np.angle(ZC_signal)

FKM_ZC = np.sin(2*np.pi*F_prom*t_one + ZC_angle) + 1j*np.cos(2*np.pi*F_prom*t_one + ZC_angle)

print(len(FKM_ZC))

: 

In [ ]:
# ФКМ сформированный ортогональной ZC-последовательностью

ZC_signal_ort = np.zeros(len(t_one), dtype=np.complex_)

for i in range(len(t_one)):
    ZC_signal_ort[i] = ZC_ort[i//n_t_sim]

ZC_angle_ort = np.angle(ZC_signal_ort)

FKM_ZC_ort = np.sin(2*np.pi*F_prom*t_one + ZC_angle_ort) + 1j*np.cos(2*np.pi*F_prom*t_one + ZC_angle_ort)

print(len(FKM_ZC_ort))

: 

In [ ]:
FKM_ZC_n = np.zeros(len(t_pach), dtype=np.complex_)

n_transmit = int(len(t_pach)//(T_p//td))

for n in np.arange(n_transmit):
    if np.divmod(n, N)[1] == 0:
        FKM_ZC_n[int(T_p//td)*n:(int(T_p//td)*n+len(FKM_ZC))] = FKM_ZC
    else:
        FKM_ZC_n[int(T_p//td)*n:(int(T_p//td)*n+len(FKM_ZC_ort))] = FKM_ZC_ort

plt.figure().set_figwidth(20)
plt.plot(t_pach, FKM_ZC_n.real)
plt.show()

: 

### Вернувшийся сигнал

In [ ]:
# вернувшийся ФКМ сформированный ZC-последовательностью

ZC_signal = np.zeros(len(t_one), dtype=np.complex_)

for i in range(len(t_one)):
    ZC_signal[i] = ZC[i//n_t_sim]

ZC_angle = np.angle(ZC_signal)

FKM_ZC_return = np.sin(2*np.pi*(F_prom+f_sm)*t_one + ZC_angle) + 1j*np.cos(2*np.pi*(F_prom+f_sm)*t_one + ZC_angle)

print(len(FKM_ZC))

: 

In [ ]:
# вернувшийся ФКМ сформированный ортогональной ZC-последовательностью

ZC_signal_ort = np.zeros(len(t_one), dtype=np.complex_)

for row_index in range(len(t_one)):
    ZC_signal_ort[row_index] = ZC_ort[row_index//n_t_sim]

ZC_angle_ort = np.angle(ZC_signal_ort)

FKM_ZC_ort_return = np.sin(2*np.pi*(F_prom+f_sm)*t_one + ZC_angle_ort) + 1j*np.cos(2*np.pi*(F_prom+f_sm)*t_one + ZC_angle_ort)

print(len(FKM_ZC_ort))

: 

In [ ]:
# шумы

noise_complex = np.random.normal(size=len(t_pach))*np.exp(-1j*2*np.pi*np.random.randn(len(t_pach)))
noise_complex = sigma_noise*noise_complex/(max(noise_complex.real) + 1j*max(noise_complex.imag))

: 

In [ ]:
# вернувшаяся пачка ФКМ

FKM_ZC_n_return = np.zeros(len(t_pach), dtype=np.complex_)

n_transmit = int((len(t_pach) - (t_z//td))//(T_p//td))

for n in np.arange(n_transmit):

    if np.divmod(n, N)[1] == 0:
        FKM_ZC_return = np.sin(2*np.pi*(F_prom+f_sm)*(t_one + T_p*n) + ZC_angle) + 1j*np.cos(2*np.pi*(F_prom+f_sm)*(t_one + T_p*n) + ZC_angle)
        FKM_ZC_n_return[int(t_z//td) + int(T_p//td)*n:int(t_z//td) + (int(T_p//td)*n+len(FKM_ZC_return))] = FKM_ZC_return
    else:
        FKM_ZC_ort_return = np.sin(2*np.pi*(F_prom+f_sm)*(t_one + T_p*n) + ZC_angle_ort) + 1j*np.cos(2*np.pi*(F_prom+f_sm)*(t_one + T_p*n) + ZC_angle_ort)
        FKM_ZC_n_return[int(t_z//td) + int(T_p//td)*n:int(t_z//td) + (int(T_p//td)*n+len(FKM_ZC_ort_return))] = FKM_ZC_ort_return

len_last = len(t_pach) - (n_transmit*int(T_p//td) + int(t_z//td))

if len_last >= len(FKM_ZC):

    if np.divmod(n_transmit, N)[1] == 0:
        FKM_ZC_return = np.sin(2*np.pi*(F_prom+f_sm)*(t_one + T_p*(n_transmit)) + ZC_angle) + 1j*np.cos(2*np.pi*(F_prom+f_sm)*(t_one + T_p*(n_transmit)) + ZC_angle)
        FKM_ZC_n_return[n_transmit*int(T_p//td) + int(t_z//td):n_transmit*int(T_p//td) + int(t_z//td) + len(FKM_ZC)] = FKM_ZC_return
    else:
        FKM_ZC_ort_return = np.sin(2*np.pi*(F_prom+f_sm)*(t_one + T_p*(n_transmit)) + ZC_angle_ort) + 1j*np.cos(2*np.pi*(F_prom+f_sm)*(t_one + T_p*(n_transmit)) + ZC_angle_ort)
        FKM_ZC_n_return[n_transmit*int(T_p//td) + int(t_z//td):n_transmit*int(T_p//td) + int(t_z//td) + len(FKM_ZC)] = FKM_ZC_ort_return
else:

    if np.divmod(n_transmit, N)[1] == 0:
        FKM_ZC_return = np.sin(2*np.pi*(F_prom+f_sm)*(t_one + T_p*(n_transmit)) + ZC_angle) + 1j*np.cos(2*np.pi*(F_prom+f_sm)*(t_one + T_p*(n_transmit)) + ZC_angle)
        FKM_ZC_n_return[n_transmit*int(T_p//td) + int(t_z//td):] = FKM_ZC_return[:len_last]
    else:
        FKM_ZC_ort_return = np.sin(2*np.pi*(F_prom+f_sm)*(t_one + T_p*(n_transmit)) + ZC_angle_ort) + 1j*np.cos(2*np.pi*(F_prom+f_sm)*(t_one + T_p*(n_transmit)) + ZC_angle_ort)
        FKM_ZC_n_return[n_transmit*int(T_p//td) + int(t_z//td):] = FKM_ZC_ort_return[:len_last]

FKM_ZC_n_return = np.array([FKM_ZC_n_return[i] if FKM_ZC_n[i] == (0.0 + 1j*0.0) else (0.0 + 1j*0.0) for i in np.arange(len(FKM_ZC_n))])

FKM_ZC_n_return = FKM_ZC_n_return + noise_complex

plt.figure().set_figwidth(20)
plt.plot(t_pach, FKM_ZC_n_return.real, "tab:orange")
plt.show()

plt.figure().set_figwidth(20)
plt.plot(t_pach, FKM_ZC_n_return.real, "tab:orange")
plt.xlim((0.66e-5, 0.68e-5))
plt.show()

plt.figure().set_figwidth(20)
plt.plot(t_pach, FKM_ZC_n_return.real, "tab:orange")
plt.xlim((2.025e-5, 2.05e-5))
plt.show()

: 

### Корреляции

In [ ]:
# lines = (FKM_ZC_n_return[x:x+len(FKM_ZC)] for x in np.arange(0, len(FKM_ZC_n_return) - len(FKM_ZC)))

ZC_corr_func = np.zeros(len(FKM_ZC_n), dtype=np.complex_)
ZC_ort_corr_func = np.zeros(len(FKM_ZC_n), dtype=np.complex_)

for x in np.arange(0, len(FKM_ZC_n_return) - len(FKM_ZC)):
    ZC_corr_func[x] = np.sum( FKM_ZC_n_return[ x : x + len(FKM_ZC) ] * np.conj( FKM_ZC ) )
    ZC_ort_corr_func[x] = np.sum( FKM_ZC_n_return[ x : x + len(FKM_ZC_ort) ] * np.conj( FKM_ZC_ort ) )


# @ray.remote
# def thread_Corr(FKM_ZC_n_return, FKM_ZC, FKM_ZC_ort, sm = 0, dx = 1):

#     ZC_corr_func = np.zeros(len(FKM_ZC_n_return), dtype=np.complex_)
#     ZC_ort_corr_func = np.zeros(len(FKM_ZC_n_return), dtype=np.complex_)

#     for x in np.arange(0 + sm, len(FKM_ZC_n_return) - len(FKM_ZC), dx):
#         FKM_ZC_corr = sp.signal.correlate(FKM_ZC, FKM_ZC_n_return[x:x+len(FKM_ZC)])
#         FKM_ZC_ort_corr = sp.signal.correlate(FKM_ZC_ort, FKM_ZC_n_return[x:x+len(FKM_ZC)])

#         ZC_corr_func[x] = max(FKM_ZC_corr)
#         ZC_ort_corr_func[x] = max(FKM_ZC_ort_corr)

#     return ZC_corr_func, ZC_ort_corr_func

# id_0 = thread_Corr.remote(FKM_ZC_n_return, FKM_ZC, FKM_ZC_ort, sm = 0, dx = 6)
# id_1 = thread_Corr.remote(FKM_ZC_n_return, FKM_ZC, FKM_ZC_ort, sm = 1, dx = 6)
# id_2 = thread_Corr.remote(FKM_ZC_n_return, FKM_ZC, FKM_ZC_ort, sm = 2, dx = 6)
# id_3 = thread_Corr.remote(FKM_ZC_n_return, FKM_ZC, FKM_ZC_ort, sm = 3, dx = 6)
# id_4 = thread_Corr.remote(FKM_ZC_n_return, FKM_ZC, FKM_ZC_ort, sm = 4, dx = 6)
# id_5 = thread_Corr.remote(FKM_ZC_n_return, FKM_ZC, FKM_ZC_ort, sm = 5, dx = 6)

# ZC_corr_func = ray.get(id_0)[0] + ray.get(id_1)[0] + ray.get(id_2)[0] + ray.get(id_3)[0] + ray.get(id_4)[0] + ray.get(id_5)[0]
# ZC_ort_corr_func = ray.get(id_0)[1] + ray.get(id_1)[1] + ray.get(id_2)[1] + ray.get(id_3)[1] + ray.get(id_4)[1] + ray.get(id_5)[1]

# ray.shutdown()


: 

In [ ]:
fig, (ax1, ax2) = plt.subplots(2)
fig.set_figwidth(20)
fig.set_figheight(10)

ax1.plot(t_pach, ZC_corr_func.real, "tab:green")
ax2.plot(t_pach, ZC_ort_corr_func.real, "tab:cyan")

# ax1.plot([(195+95)*dt_z for i in range(0, 1000, 100)], range(0, 1000, 100))

: 

### Сборка корреляций в столбцы по периодам

In [ ]:
Corr_Matrix_1 = np.zeros( ( int( len( t_pach ) // int( T_p // td ) ), int( T_p // td ) ), np.complex_ )         # Случай приема всех 5 импульсов обратно, соответственно собираю все 5 поочередно с первого вернувшегося (с зеленого)
Corr_Matrix_2 = np.zeros( ( int( len( t_pach ) // int( T_p // td ) ) - 1, int( T_p // td ) ), np.complex_ )     # Сулчай приема только 4 вернувшихся импульсов, соответственно собираю 4 также поочередно с первого вернувшегося (тоже с зеленого)

for row_index in np.arange( int( len( t_pach ) // int( T_p // td ) ) ):
    if row_index == 0:
            Corr_Matrix_1[row_index] = ZC_corr_func[ row_index * int( T_p // td ) : ( row_index + 1 ) * int( T_p // td ) ]

    else:
        if np.divmod(row_index, N)[1] == 0: # Миша, это просто mod() = np.divmod()[1]!
            Corr_Matrix_1[row_index] = ZC_corr_func[ row_index * int( T_p // td ) : ( row_index + 1 ) * int( T_p // td ) ]       

        else:
            Corr_Matrix_1[row_index] = ZC_ort_corr_func[ row_index * int( T_p // td ) : ( row_index + 1 ) * int( T_p // td ) ]



        if np.divmod(row_index - 1, N)[1] == 0:
            Corr_Matrix_2[row_index - 1] = ZC_corr_func[ row_index * int( T_p // td ) : ( row_index + 1 ) * int( T_p // td ) ]

        else:
            Corr_Matrix_2[row_index - 1] = ZC_ort_corr_func[ row_index * int( T_p // td ) : ( row_index + 1 ) * int( T_p // td ) ]

maximum_level_1 = np.amax( Corr_Matrix_1 )
maximum_level_2 = np.amax( Corr_Matrix_2 )
abs_maximum_level = np.amax( ( maximum_level_1, maximum_level_2 ) )

print("Абсолютный максимум среди корреляций:", abs_maximum_level )

fig1, ax1 = plt.subplots( int( len( t_pach ) // int( T_p // td ) ), 2 )

fig1.set_figheight(3 * len(ax1))
fig1.set_figwidth(15)

for i in range(len(ax1)):
    Corr_Matrix_1[i] = Corr_Matrix_1[i] / abs_maximum_level
    ax1[i][0].plot(range(len(Corr_Matrix_1[i])), Corr_Matrix_1[i].real )

    if i < len(ax1) - 1:
        Corr_Matrix_2[i] = Corr_Matrix_2[i] / abs_maximum_level
        ax1[i][1].plot(range(len(Corr_Matrix_2[i])), Corr_Matrix_2[i].real )

    ax1[i][0].set_ylim((-1.2, 1.2))
    ax1[i][1].set_ylim((-1.2, 1.2))

: 

### Децимация матриц по дальности (времени)

In [ ]:
dR_Matrix_1 = np.zeros( ( len(Corr_Matrix_1), int( T_p // td ) // int( dt_z // td ) ), np.complex_ )
dR_Matrix_2 = np.zeros( ( len(Corr_Matrix_2), int( T_p // td ) // int( dt_z // td ) ), np.complex_ )

ratio = len(Corr_Matrix_1[0])//len(dR_Matrix_1[0])
# smeshen = 10000

print("Отношение размеров корреляционной и матрицы по каналам: ", ratio)

for row_index in np.arange( len( dR_Matrix_1 ) ):
    if row_index == 0:
            dR_Matrix_1[row_index] = [ np.sum( Corr_Matrix_1[ row_index ][ col_index * ratio : (col_index+1) * ratio ] \
                * np.exp( -1j * 2 * np.pi * F_prom * (td * np.abs( col_index * ratio - np.arange( col_index * ratio, (col_index+1) * ratio ) ) + T_p * row_index ) ) \
                    ) for col_index in np.arange( len( dR_Matrix_1[ row_index ] ) ) ]

    else:
        dR_Matrix_1[row_index] = [ np.sum( Corr_Matrix_1[ row_index ][ col_index * ratio : (col_index+1) * ratio ] \
                * np.exp( -1j * 2 * np.pi * F_prom * (td * np.abs( col_index * ratio - np.arange( col_index * ratio, (col_index+1) * ratio ) ) + T_p * row_index ) ) \
                    ) for col_index in np.arange( len( dR_Matrix_1[ row_index ] ) ) ]

        dR_Matrix_2[row_index - 1] = [ np.sum( Corr_Matrix_2[ row_index - 1 ][ col_index * ratio : (col_index+1) * ratio ] \
                * np.exp( -1j * 2 * np.pi * F_prom * (td * np.abs( col_index * ratio - np.arange( col_index * ratio, (col_index+1) * ratio ) ) + T_p * row_index ) ) \
                    ) for col_index in np.arange( len( dR_Matrix_2[ row_index - 1 ] ) ) ]

# print(len(Corr_Matrix_1[0]))
# print(Corr_Matrix_1[0, smeshen:smeshen+ratio])

# plt.plot(Corr_Matrix_1[0, smeshen:smeshen+ratio].real, Corr_Matrix_1[0, smeshen:smeshen+ratio].imag, "bo")
# plt.show()

# Corr_Matrix_1[0, smeshen:smeshen+ratio] = Corr_Matrix_1[0, smeshen:smeshen+ratio] * np.exp(-1j * 2 * np.pi * F_prom * td * (0 - np.arange(smeshen,smeshen+ratio)))
# # Corr_Matrix_1[1, 0:20] = Corr_Matrix_1[1, 0:20] * np.exp(-1j * 2 * np.pi * F_prom * td * (0 - np.arange(0,20)))
# # Corr_Matrix_1[2, 0:20] = Corr_Matrix_1[2, 0:20] * np.exp(-1j * 2 * np.pi * F_prom * td * (0 - np.arange(0,20)))
# # Corr_Matrix_1[3, 0:20] = Corr_Matrix_1[3, 0:20] * np.exp(-1j * 2 * np.pi * F_prom * td * (0 - np.arange(0,20)))
# # Corr_Matrix_1[4, 0:20] = Corr_Matrix_1[4, 0:20] * np.exp(-1j * 2 * np.pi * F_prom * td * (0 - np.arange(0,20)))

# # print(len(Corr_Matrix_1[0]))
# # print(Corr_Matrix_1[0, smeshen:smeshen+ratio])

# plt.plot(Corr_Matrix_1[0, smeshen:smeshen+ratio].real, Corr_Matrix_1[0, smeshen:smeshen+ratio].imag, "bo")
# plt.show()


: 

In [ ]:
maximum_level_1 = np.amax( np.abs(dR_Matrix_1) )
maximum_level_2 = np.amax( np.abs(dR_Matrix_2) )
abs_maximum_level = np.amax( ( maximum_level_1, maximum_level_2 ) )

print("Абсолютный максимум каналов дальности (задержки):", abs_maximum_level )

fig1, ax1 = plt.subplots( len( dR_Matrix_1 ), 2 )

fig1.set_figheight(3 * len(ax1))
fig1.set_figwidth(15)

for i in range(len(ax1)):
    dR_Matrix_1[i] = dR_Matrix_1[i]/abs_maximum_level
    # ax1[i][0].plot(range(len(dR_Matrix_1[i])), dR_Matrix_1[i].real )
    ax1[i][0].plot(range(len(dR_Matrix_1[i])), np.abs(dR_Matrix_1[i]) )

    if i < len(ax1) - 1:
        dR_Matrix_2[i] = dR_Matrix_2[i]/abs_maximum_level
        ax1[i][1].plot(range(len(dR_Matrix_2[i])), np.abs(dR_Matrix_2[i]) )

    ax1[i][0].set_ylim((0, 1.2))
    ax1[i][1].set_ylim((0, 1.2))

: 

### FFT по столбцам (переходим к каналам по частоте-скорости)

In [ ]:
dRF_Matrix_1 = np.copy(dR_Matrix_1)
dRF_Matrix_2 = np.copy(dR_Matrix_2)

# ---------------------------------------------------------------------------------------------

# весовое окно по частоте

window_1 = 1 - 0.8 * np.cos(2*np.pi*(np.arange(len( dRF_Matrix_1 ))+0.5)/len( dRF_Matrix_1 ))
window_2 = 1 - 0.8 * np.cos(2*np.pi*(np.arange(len( dRF_Matrix_2 ))+0.5)/len( dRF_Matrix_2 ))

plt.figure().set_figwidth(20)
plt.plot(window_1, "bo")
plt.plot(window_2, "ro")
plt.show()

# for col_index in np.arange( len( dRF_Matrix_1[0] ) ):
#     dRF_Matrix_1[ : , col_index] = dRF_Matrix_1[ : , col_index] * window_1
#     dRF_Matrix_2[ : , col_index] = dRF_Matrix_2[ : , col_index] * window_2

# ---------------------------------------------------------------------------------------------

for col_index in np.arange( len( dRF_Matrix_1[0] ) ):
    dRF_Matrix_1[ : , col_index] = sp.fft.fft( dRF_Matrix_1[ : , col_index] )
    dRF_Matrix_2[ : , col_index] = sp.fft.fft( dRF_Matrix_2[ : , col_index] )

maximum_level_1 = np.amax( np.abs(dRF_Matrix_1) )
maximum_level_2 = np.amax( np.abs(dRF_Matrix_2) )
abs_maximum_level = np.amax( ( maximum_level_1, maximum_level_2 ) )

print("Абсолютный максимум каналов дальности (задержки):", abs_maximum_level )

dRF_Matrix_1 = dRF_Matrix_1/abs_maximum_level
dRF_Matrix_2 = dRF_Matrix_2/abs_maximum_level

fig1, ax1 = plt.subplots(subplot_kw={"projection": "3d"})

fig1.set_figheight(8)
fig1.set_figwidth(8)

# Make data.
X1 = np.arange(0, len(dRF_Matrix_1[0]))
Y1 = np.arange(0, len(dRF_Matrix_1))
X1, Y1 = np.meshgrid(X1, Y1)
Z1 = np.abs(dRF_Matrix_1)

# Plot the surface.
surf1 = ax1.plot_surface(X1, Y1, Z1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

ax1.zaxis.set_major_locator(LinearLocator(13))
ax1.zaxis.set_major_formatter('{x:.1f}')
ax1.set_zlim((-0.2, 1.2))

ax1.xaxis.set_major_locator(LinearLocator(6))
ax1.xaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax1.set_xlim((0, 195))
ax1.set_xlabel("Каналы по дальности, dR")

ax1.yaxis.set_major_locator(LinearLocator(6))
ax1.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax1.set_ylim((0, 5))
ax1.set_ylabel("Каналы по скорости, dV")

# Add a color bar which maps values to colors.
# fig1.colorbar(surf1, shrink=0.5, aspect=5)


fig2, ax2 = plt.subplots(subplot_kw={"projection": "3d"})

fig2.set_figheight(8)
fig2.set_figwidth(8)

# Make data.
X2 = np.arange(0, len(dRF_Matrix_2[0]))
Y2 = np.arange(0, len(dRF_Matrix_2))
X2, Y2 = np.meshgrid(X2, Y2)
Z2 = np.abs(dRF_Matrix_2)

# Plot the surface.
surf2 = ax2.plot_surface(X2, Y2, Z2, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

ax2.zaxis.set_major_locator(LinearLocator(13))
ax2.zaxis.set_major_formatter('{x:.1f}')
ax2.set_zlim((-0.2, 1.2))

ax2.xaxis.set_major_locator(LinearLocator(6))
ax2.xaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax2.set_xlim((0, 195))
ax2.set_xlabel("Каналы по дальности, dR")

ax2.yaxis.set_major_locator(LinearLocator(5))
ax2.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax2.set_ylim((0, 4))
ax2.set_ylabel("Каналы по скорости, dV")

# Add a color bar which maps values to colors.
# fig2.colorbar(surf2, shrink=0.5, aspect=5)

: 

In [ ]:
max_R_1 = []
for i in range(len(dRF_Matrix_1)):
    max_R_1.append([np.argmax( np.abs( dRF_Matrix_1[i] ) ),  np.abs(dRF_Matrix_1[i, np.argmax( np.abs( dRF_Matrix_1[i] ) )])])

max_R_1 = np.array(max_R_1)

max_RV_1 = np.argmax(max_R_1[:, 1])

print(max_RV_1)
print(max_R_1[max_RV_1][0])
print("\n")

max_R_2 = []
for i in range(len(dRF_Matrix_2)):
    max_R_2.append([np.argmax( np.abs( dRF_Matrix_2[i] ) ),  np.abs(dRF_Matrix_2[i, np.argmax( np.abs( dRF_Matrix_2[i] ) )])])

max_R_2 = np.array(max_R_2)

max_RV_2 = np.argmax(max_R_2[:, 1])

print(max_RV_2)
print(max_R_2[max_RV_2][0])
print("\n")

: 

In [ ]:
plt.figure().set_figwidth(10)
plt.plot(np.abs(dRF_Matrix_1[max_RV_1]))
plt.plot([0.5 for i in range(len(dRF_Matrix_1[max_RV_1]))], linestyle="-.")
plt.plot([max_R_1[max_RV_1][0], max_R_1[max_RV_1][0]], [-0.2, 1.2], linestyle="-.")
plt.plot([0 for i in range(len(dRF_Matrix_1[max_RV_1]))], linestyle="-")
plt.ylim((-0.2, 1.2))
plt.xlim((0, 200))
plt.xlabel("Каналы дальности, dR")
plt.show()

plt.figure().set_figwidth(10)
plt.plot(np.abs(dRF_Matrix_2[max_RV_2]))
plt.plot([0.5 for i in range(len(dRF_Matrix_2[max_RV_2]))], linestyle="-.")
plt.plot([max_R_2[max_RV_2][0], max_R_2[max_RV_2][0]], [-0.2, 1.2], linestyle="-.")
plt.plot([0 for i in range(len(dRF_Matrix_2[max_RV_2]))], linestyle="-")
plt.ylim((-0.2, 1.2))
plt.xlim((0, 200))
plt.xlabel("Каналы дальности, dR")
plt.show()

: 

In [ ]:
plt.figure().set_figwidth(10)
plt.plot(np.abs(dRF_Matrix_1[:, int(max_R_1[max_RV_1][0])]), "bo")
plt.plot(np.abs(dRF_Matrix_1[:, int(max_R_1[max_RV_1][0])]))
plt.plot([0.5 for i in range(len(dRF_Matrix_1[:, int(max_R_1[max_RV_1][0])]))], "tab:orange", linestyle="-.")
plt.plot([0 for i in range(len(dRF_Matrix_1[:, int(max_R_1[max_RV_1][0])]))], "tab:purple", linestyle="-")
plt.ylim((-0.2, 1.2))
plt.xlim((0, 4))
plt.xlabel("Каналы скорости, dV")
plt.show()

plt.figure().set_figwidth(10)
plt.plot(np.abs(dRF_Matrix_2[:, int(max_R_2[max_RV_2][0])]), "bo")
plt.plot(np.abs(dRF_Matrix_2[:, int(max_R_2[max_RV_2][0])]))
plt.plot([0.5 for i in range(len(dRF_Matrix_2[:, int(max_R_2[max_RV_2][0])]))], "tab:orange", linestyle="-.")
plt.plot([0 for i in range(len(dRF_Matrix_2[:, int(max_R_2[max_RV_2][0])]))], "tab:purple", linestyle="-")
plt.ylim((-0.2, 1.2))
plt.xlim((0, 3))
plt.xlabel("Каналы скорости, dV")
plt.show()

: 

In [ ]:
data = np.abs(dRF_Matrix_2[:, int(max_R_2[max_RV_2][0])])
print(data)

x = np.array([0, 1, 2])
data1 = np.polyfit(x, data[:-1], 2)

plt.figure().set_figwidth(10)
plt.plot(np.abs(dRF_Matrix_2[:, int(max_R_2[max_RV_2][0])]), "bo")
plt.plot(np.abs(dRF_Matrix_2[:, int(max_R_2[max_RV_2][0])]))
plt.plot(data1, "ro")
plt.plot(data1, "r-")
plt.plot([0.5 for i in range(len(dRF_Matrix_2[:, int(max_R_2[max_RV_2][0])]))], "tab:orange", linestyle="-.")
plt.plot([0 for i in range(len(dRF_Matrix_2[:, int(max_R_2[max_RV_2][0])]))], "tab:purple", linestyle="-")
plt.ylim((-0.2, 1.2))
plt.xlim((0, 3))
plt.xlabel("Каналы скорости, dV")
plt.show()

: 